In [ ]:
%pip install plotly
%pip install nbformat

In [2]:
from kafka.admin import KafkaAdminClient, NewTopic
from kafka.structs import TopicPartition

import plotly.express as px
import json
import pandas as pd

In [24]:
from kafka import KafkaConsumer

# auto_offset_reset & group_id are CRUCIAL for it to work. No idea why
consumer = KafkaConsumer('energy_api', consumer_timeout_ms=10000, bootstrap_servers = 'localhost:9092', auto_offset_reset='earliest', group_id=None)

dfs = []

partitions = consumer.partitions_for_topic('energy_api')

for p in partitions:
    topic_partition = TopicPartition('energy_api', p)
    # Seek offset 0
    # consumer.seek(partition=topic_partition, offset=0)
    for msg in consumer:
        try:
            dict = json.loads(msg.value)
            df = pd.DataFrame.from_records(dict)
            dfs.append(df)
            
        except:
            print(msg.value[:20])
            continue
df = pd.concat(dfs, ignore_index=True) # concatenate all the data frames in the list.
df

b'TEST'
b'TEST2'


,Minutes5UTC,Minutes5DK,PriceArea,ProductionLt100MW,ProductionGe100MW,OffshoreWindPower,OnshoreWindPower,SolarPower,ExchangeGreatBelt,ExchangeGermany,ExchangeNetherlands,ExchangeNorway,ExchangeSweden,BornholmSE4
0,2023-02-06T09:20:00,2023-02-06T10:20:00,DK1,594.280029,1022.679993,-7.280000,28.850000,218.990005,-354.000000,-568.909973,700.380005,1107.680054,236.000000,NaN
1,2023-02-06T09:20:00,2023-02-06T10:20:00,DK2,195.460007,548.739990,31.840000,25.070000,77.379997,354.000000,-421.899994,NaN,NaN,974.669983,32.049999
2,2023-02-06T09:25:00,2023-02-06T10:25:00,DK1,592.340027,1012.020020,-6.270000,22.940001,234.770004,-354.000000,-558.880005,700.380005,1107.339966,236.000000,NaN
3,2023-02-06T09:25:00,2023-02-06T10:25:00,DK2,198.199997,551.010010,24.440001,24.740000,76.830002,354.000000,-421.799988,NaN,NaN,1017.760010,33.689999
4,2023-02-06T09:30:00,2023-02-06T10:30:00,DK1,597.530029,993.780029,-2.800000,20.780001,235.080002,-354.000000,-538.559998,700.380005,1107.770020,236.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,2023-02-03T18:15:00,2023-02-03T19:15:00,DK2,180.910004,497.049988,910.369995,401.149994,-0.220000,-337.769989,-382.769989,NaN,NaN,499.779999,-10.820000
108,2023-02-03T18:35:00,2023-02-03T19:35:00,DK1,265.899994,613.409973,1096.869995,1362.599976,5.360000,334.250000,573.330017,-700.429993,-690.000000,214.000000,NaN
109,2023-02-03T18:35:00,2023-02-03T19:35:00,DK2,179.250000,479.369995,935.940002,381.649994,-0.240000,-334.250000,-381.489990,NaN,NaN,469.579987,-10.750000
110,2023-02-06T09:35:00,2023-02-06T10:35:00,DK1,600.659973,986.190002,-0.590000,17.170000,237.110001,-354.000000,-543.590027,700.380005,1107.670044,236.000000,NaN


In [ ]:
df["Minutes5DK"] = pd.to_datetime(df["Minutes5DK"])
df["Minutes5UTC"] = pd.to_datetime(df["Minutes5UTC"])

In [ ]:
avg_by_minute = df.groupby(df.Minutes5DK).mean()
avg_by_minute

/tmp/ipykernel_17291/493087497.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  avg_by_minute = df.groupby(df.Minutes5DK).mean()


,ProductionLt100MW,ProductionGe100MW,OffshoreWindPower,OnshoreWindPower,SolarPower,ExchangeGreatBelt,ExchangeGermany,ExchangeNetherlands,ExchangeNorway,ExchangeSweden,BornholmSE4
Minutes5DK,,,,,,,,,,,
2023-02-05 16:20:00,272.690002,713.649994,586.270004,572.214970,34.605000,0.0,-547.839997,-646.130005,482.480011,434.820007,6.62
2023-02-05 16:25:00,272.429993,732.984985,577.854996,544.319992,30.280000,0.0,-518.699997,-646.130005,483.489990,439.464996,7.61
2023-02-05 16:30:00,274.644996,741.050018,582.299988,529.620003,22.830000,0.0,-505.309998,-646.130005,484.049988,443.309998,6.75
2023-02-05 16:35:00,277.295006,743.570007,570.980011,527.239994,18.264999,0.0,-504.899994,-646.130005,484.070007,469.755005,7.44
2023-02-05 16:40:00,279.835007,744.734985,562.394989,508.864986,14.715000,0.0,-507.010010,-646.130005,483.440002,519.609985,7.98


In [ ]:
px.line(data_frame = avg_by_minute, x=avg_by_minute.index, y=["OffshoreWindPower", "OnshoreWindPower", "SolarPower"])

In [ ]:
px.line(data_frame = avg_by_minute, x=avg_by_minute.index, y=["ExchangeGermany", "ExchangeNetherlands", "ExchangeNorway", "ExchangeSweden"])
